# Table of Content #

- [Importing Necessary Libraries](#Importing-Necessary-Libraries)
- [Importing Data and Initial Checks](#Importing-Data-and-Initial-Checks)
- [Target Variable and Features Matrix](#Target-Variable-and-Features-Matrix)

## Importing Necessary Libraries ##

In [1]:
import pandas as pd
import numpy as np


import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('fivethirtyeight')

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from sklearn.pipeline                import Pipeline
from sklearn.model_selection         import train_test_split, GridSearchCV
from sklearn.linear_model            import LogisticRegression
from sklearn.ensemble                import BaggingClassifier,RandomForestClassifier,ExtraTreesClassifier,AdaBoostClassifier,GradientBoostingClassifier

from sklearn.tree                    import DecisionTreeClassifier
from sklearn.svm                     import SVC

import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

from keras.callbacks import EarlyStopping
from keras.layers import Dropout

Using TensorFlow backend.


## Importing Data and Initial Checks ##

In [2]:
#Loading data from a csv file
data = pd.read_csv('~/ga/projects/capstone_data/data/data_ready.csv')

#Checking size
data.shape

(644232, 12)

In [3]:
#Checking columns
data.columns

Index(['Unnamed: 0', 'month', 'day_of_month', 'day_of_week',
       'op_carrier_fl_num', 'origin', 'dest', 'arr_delay', 'delay_indicator',
       'distance', 'carrier', 'dep_hour'],
      dtype='object')

In [4]:
#Dropping a technical column
data.drop(columns = ['Unnamed: 0'], axis=1, inplace=True)

#Checking DataFrame
data.head()

,month,day_of_month,day_of_week,op_carrier_fl_num,origin,dest,arr_delay,delay_indicator,distance,carrier,dep_hour
0,10,3,3,5228,ONT,SFO,-12.0,0.0,363.0,Delta,11
1,11,7,3,1443,BNA,DAL,-7.0,0.0,623.0,SouthWest,15
2,12,14,5,4072,LGA,CLE,-12.0,0.0,419.0,United,15
3,12,9,7,331,JFK,LAX,-17.0,0.0,2475.0,American,11
4,12,17,1,3539,SLC,GEG,-19.0,0.0,546.0,Delta,15


## Target Variable and Features Matrix ##

In order to fit a logistic regression we need to use our **DELAY_INDICATOR** as our target variable. We also need to drop ARR_DELAY from our features as our target variable was efficiently engineered from it.

In [5]:
#Target variable
y = data['delay_indicator']

#Features matrix
X = data.drop(columns=['delay_indicator','arr_delay'])

In [6]:
#Baseline model accuracy
y.mean()

0.5

In [7]:
#Checking our feature matrix data types
X.dtypes

month                  int64
day_of_month           int64
day_of_week            int64
op_carrier_fl_num      int64
origin                object
dest                  object
distance             float64
carrier               object
dep_hour               int64
dtype: object

In [8]:
#Getting dummies for our text features ORIGIN, DEST and CARRIER
X = pd.get_dummies(X,columns = ['origin','dest','carrier'],drop_first=True)

#Checking the shape of our feature matrix
X.shape

(644232, 713)

In [9]:
#Training and testing sets split with random_state=1519 for reproduceability of results 
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1519)

In [10]:
#As an additional step, we need to scale our feature matrices modeling
ss = StandardScaler()
X_train = pd.DataFrame(ss.fit_transform(X_train),columns = X.columns)
# X_train = ss.fit_transform(X_train)
X_test = pd.DataFrame(ss.transform(X_test),columns = X.columns)

## FFNN ##

In [11]:
model = Sequential()

model.add(Dense(128,
         input_shape = (713,),
         activation = 'relu'))

# model.add(Dropout(0.25))

model.add(Dense(64,
               activation='relu'))

# model.add(Dropout(0.25))

model.add(Dense(1,
               activation = 'sigmoid'))

early_stop = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=5, verbose=1, mode='auto')

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy']
             )

In [12]:
history = model.fit(
    X_train,
    y_train,
    validation_data=(X_test, y_test),
    epochs=100,
    batch_size=512,
    verbose=2,
    callbacks=[early_stop]
)

Train on 483174 samples, validate on 161058 samples
Epoch 1/100
483174/483174 - 24s - loss: 0.6722 - accuracy: 0.5877 - val_loss: 0.6634 - val_accuracy: 0.5985
Epoch 2/100
483174/483174 - 23s - loss: 0.6574 - accuracy: 0.6063 - val_loss: 0.6573 - val_accuracy: 0.6061
Epoch 3/100
483174/483174 - 32s - loss: 0.6508 - accuracy: 0.6160 - val_loss: 0.6535 - val_accuracy: 0.6113
Epoch 4/100
483174/483174 - 38s - loss: 0.6453 - accuracy: 0.6226 - val_loss: 0.6503 - val_accuracy: 0.6162
Epoch 5/100
483174/483174 - 25s - loss: 0.6388 - accuracy: 0.6305 - val_loss: 0.6451 - val_accuracy: 0.6228
Epoch 6/100
483174/483174 - 28s - loss: 0.6320 - accuracy: 0.6388 - val_loss: 0.6413 - val_accuracy: 0.6274
Epoch 7/100
483174/483174 - 25s - loss: 0.6265 - accuracy: 0.6449 - val_loss: 0.6400 - val_accuracy: 0.6294
Epoch 8/100
483174/483174 - 24s - loss: 0.6214 - accuracy: 0.6500 - val_loss: 0.6387 - val_accuracy: 0.6319
Epoch 9/100
483174/483174 - 25s - loss: 0.6169 - accuracy: 0.6545 - val_loss: 0.6379